## Importar lo necesario

In [ ]:
import torch
import torchvision
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib import image
from matplotlib import pyplot


# Ejercicio - cálculo de dimensiones

Implemente con las clases de `conv2d` y `MaxPool2d` los siguientes casos:

#### ejercicio a)
Implemente una capa de convolución de 1 canal de entrada y salida, con kernel de tamaño 5 y un padding que conserve el tamaño de la imagen; seguido de una capa de pooling que reduzca en la mitad las dimensiones de la imagen.

In [ ]:
conv = torch.nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 5, padding ='same')
pool = torch.nn.MaxPool2d(kernel_size = 2, stride=2, padding=0)

imagen_ejemplo= torch.rand(1, 1, 100, 100)
# dimensiones: (N, #filtros, H_out, W_out)
print('tamaño imagen ejemplo')
print(imagen_ejemplo.shape)

# nuestro "forward"
x = conv(imagen_ejemplo)
print('tamaño salida conv')
print(x.shape)

salida = pool(x)

print('tamaño imagen salida')
print(salida.shape)

tamaño imagen ejemplo
torch.Size([1, 1, 100, 100])
tamaño salida conv
torch.Size([1, 1, 100, 100])
tamaño imagen salida
torch.Size([1, 1, 50, 50])


#### ejercicio b)
Implemente una capa de convolución de 10 canales con kernel de tamaño 5, un padding de 0 y stride para que el kernel no se pise a si mismo.

¿cual es le tamaño final de la salida, si entra una imagen de 200x100x3?

In [ ]:
conv = torch.nn.Conv2d(in_channels = 3, out_channels = 10, kernel_size = 5, padding = 0, stride=5)

imagen_ejemplo= torch.rand(1, 3, 200, 100)
# dimensiones: (N, #filtros, H_out, W_out)
print('tamaño imagen ejemplo')
print(imagen_ejemplo.shape)

# nuestro "forward"
salida = conv(imagen_ejemplo)

print('tamaño imagen salida')
print(salida.shape)

tamaño imagen ejemplo
torch.Size([1, 3, 200, 100])
tamaño imagen salida
torch.Size([1, 10, 40, 20])


#### ejercicio c)
Implemente 3 capas de convolución en cascada, cada una con kernel size de 7, numero de canales de salida de [8, 16, 32] respectivamente y, con padding de 0. Al final, coloque una capa de pooling con kernel del 7, con un stride para que no se pisen a si mismo.

¿cual es le tamaño final de la salida, si entra una imagen de 500x500x1?

¿cual es el número de parámetros entenables?

In [ ]:
conv1 = torch.nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = 7, padding = 0) #pierde 3+3 , 3+3
conv2 = torch.nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 7, padding = 0) #pierde 3+3 , 3+3
conv3 = torch.nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 7, padding = 0) #pierde 3+3 , 3+3
pool = torch.nn.MaxPool2d(kernel_size = 7, stride=7, padding=0, ceil_mode=False) # reduce en 7 veces, ver ceil mode!

imagen_ejemplo= torch.rand(1, 1, 500, 500)
# dimensiones: (N, #filtros, H_out, W_out)
print('tamaño imagen ejemplo')
print(imagen_ejemplo.shape)

# nuestro "forward"
x = conv1(imagen_ejemplo)
x = conv2(x)
x = conv3(x)
print('tamaño parcial ANTES del pooling')
print(x.shape)
salida = pool(x)

print('tamaño imagen salida')
print(salida.shape)

print('tamaño de parámetros')
print(conv1.weight.shape)
print(conv2.weight.shape)
print(conv3.weight.shape)

print('tamaño de bias')
print(conv1.bias.shape)
print(conv2.bias.shape)
print(conv3.bias.shape)

tamaño imagen ejemplo
torch.Size([1, 1, 500, 500])
tamaño parcial ANTES del pooling
torch.Size([1, 32, 482, 482])
tamaño imagen salida
torch.Size([1, 32, 68, 68])
tamaño de parámetros
torch.Size([8, 1, 7, 7])
torch.Size([16, 8, 7, 7])
torch.Size([32, 16, 7, 7])
tamaño de bias
torch.Size([8])
torch.Size([16])
torch.Size([32])


#### ejercico d)
Si tiene una imagen a procesar de 300x200x3 y desea aplicar 3 capas de convolución+pooling en cascada con las siguientes características:

* kernel convolución (común a todas las capas) = 5
* padding convolución (común a todas las capas) = 0
* stride convolución (común a todas las capas) = 1
* canales conv1 = 16
* canales conv2 = 32
* canales conv3 = 64

* kernel pooling = 2
* padding pooling = 0
* stride pooling = 2

¿cual es el tamaño final de la salida y cuantos canales tiene?

Si tiene que conectar esa salida a una `fully_connected` para clasificar 10 clases ¿que tamaño debe tener?

¿Cúal es el número total de parámetros a entrenar (contando la `fc`)?

In [ ]:
conv1 = torch.nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 5, padding = 0) #pierde 2+2 , 2+2
conv2 = torch.nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5, padding = 0) #pierde 2+2 , 2+2
conv3 = torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 5, padding = 0) #pierde 2+2 ,2+2

pool = torch.nn.MaxPool2d(kernel_size = 2, stride=2, padding=0, ceil_mode=False) # reduce en 2 veces, ver ceil mode!

imagen_ejemplo= torch.rand(1, 3, 300,200)
# dimensiones: (N, #filtros, H_out, W_out)
print('tamaño imagen ejemplo')
print(imagen_ejemplo.shape)

# nuestro "forward"
x = conv1(imagen_ejemplo)
x = pool(x)
x = conv2(x)
x = pool(x)
x = conv3(x)
x = pool(x)
print('tamaño parcial ANTES de fc')
forma = x.shape
print(forma)

# agrego la fc

fc = torch.nn.Linear(in_features = forma[1]*forma[2]*forma[3], out_features = 10)
salida = fc(x.view(x.shape[0], -1))
print()
print('salida final de la fc')
print(salida.shape)

# numero de parametros
total = torch.tensor([0])
print('--'*20)
print('--'*20)
for layer in [conv1, conv2, conv3, fc]:
  kernel = torch.tensor(layer.weight.shape)
  bias = torch.tensor(layer.bias.shape)
  print('layer: ')
  print(layer)
  print('kernel o parámetros')
  print(kernel)
  print('bias')
  print(bias)
  print('total')
  total += torch.prod(kernel) + bias
  print(total)
  print('------')


tamaño imagen ejemplo
torch.Size([1, 3, 300, 200])
tamaño parcial ANTES de fc
torch.Size([1, 64, 34, 21])

salida final de la fc
torch.Size([1, 10])
----------------------------------------
----------------------------------------
layer: 
Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
kernel o parámetros
tensor([16,  3,  5,  5])
bias
tensor([16])
total
tensor([1216])
------
layer: 
Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
kernel o parámetros
tensor([32, 16,  5,  5])
bias
tensor([32])
total
tensor([14048])
------
layer: 
Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
kernel o parámetros
tensor([64, 32,  5,  5])
bias
tensor([64])
total
tensor([65312])
------
layer: 
Linear(in_features=45696, out_features=10, bias=True)
kernel o parámetros
tensor([   10, 45696])
bias
tensor([10])
total
tensor([522282])
------
